In [ ]:
!wget https://github.com/MorvanZhou/PyTorch-Tutorial/raw/master/tutorial-contents-notebooks/mnist/processed/training.pt
!wget https://github.com/MorvanZhou/PyTorch-Tutorial/raw/master/tutorial-contents-notebooks/mnist/processed/test.pt

--2021-09-29 10:42:00--  https://github.com/MorvanZhou/PyTorch-Tutorial/raw/master/tutorial-contents-notebooks/mnist/processed/training.pt
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/MorvanZhou/PyTorch-Tutorial/master/tutorial-contents-notebooks/mnist/processed/training.pt [following]
--2021-09-29 10:42:00--  https://raw.githubusercontent.com/MorvanZhou/PyTorch-Tutorial/master/tutorial-contents-notebooks/mnist/processed/training.pt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 47520431 (45M) [application/octet-stream]
Saving to: ‘training.pt’

training.pt         100%[===================>]

In [ ]:
!pip install pytorch-lightning

     |████████████████████████████████| 924 kB 5.4 MB/s 
     |████████████████████████████████| 636 kB 39.5 MB/s 
     |████████████████████████████████| 829 kB 48.7 MB/s 
     |████████████████████████████████| 123 kB 45.5 MB/s 
     |████████████████████████████████| 282 kB 52.2 MB/s 
     |████████████████████████████████| 1.3 MB 41.0 MB/s 
     |████████████████████████████████| 294 kB 47.2 MB/s 
     |████████████████████████████████| 142 kB 50.7 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=ff6c6a9d4b9b84e811018317e97737a91f2d81d41b9daa21348f9cbddcc45b67
  Stored in directory: /root/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
Successfully built future
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall: future
    Found existing installation: future 0.16.0
    Uninstalling future-0

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, random_split
from torch.optim import Adam
import pytorch_lightning
from torchvision import transforms, datasets

In [ ]:
device = torch.device('cuda')

In [ ]:
GPU_COUNT = min(1, torch.cuda.device_count())
BATCH_SIZE = 256 if torch.cuda.is_available() else 64

In [ ]:
train_data, val_data = random_split(TensorDataset(full_training_data), [40000,20000])

AttributeError: ignored

In [ ]:
train_loader = DataLoader()

In [ ]:
class MNIST_Lightning(pl.LightningModule):

    def __innit__(self, train_loader, val_loader=None, test_loader = None, learning_rate=0.001):
        super(MNIST_Lightning, self).__init__()

        self.train_loader = train_loader
        self.val_loader = val_loader
        self.test_loader = test_loader
        self.learning_rate = learning_rate
        self.h1 = nn.Linear(in_features=28*28, out_features=128)
        self.out = nn.Linear(in_features=128, out_features=10)

    def forward(self, x):

        x = x.view(x.size()[0],-1)
        x = self.h1(x)
        x = F.relu(x)
        x = self.out(x)
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        output = torch.softmax(self(x), dim=1)
        loss = F.cross_entropy(output, y)
        return loss

    def validation_step(self, batch, batch_idx):
        x,y = batch
        output = torch.softmax(self(x), dim=1)
        loss = F.cross_entropy(output, y)
        pred = torch.argmax(output, dim=1)
        acc = accuracy(pred, y)

        self.log('Validation loss', loss, prog_bar = True)
        self.log('Validation accuracy', acc, prog_bar=True)

        return loss

        def test_step(self, batch, batch_idx):
            return self.validation_step(batch, batch_idx)

        def train_dataloader(self):
            return self.train_loader
        
        def val_dataloader(self):
            return self.val_loader

        def test_dataloader(self):
            return self.test_loader

        def configure_optimizers(self):
            return Adm(self.parameters(), lr = self.learning_rate)


In [ ]:
model = MNIST_Lightning(train_loader, val_loader=val_loader, test_loader=test_loader)
trainer = pl.Trainer(
    gpus = GPU_COUNT,
    max_epochs = 50,
    progress_bar_reresh_rate=20
) 

In [ ]:
trainer.fit(model)